Импортируем библиотеки

In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import catboost as cb
from datetime import datetime
import seaborn as sns

In [144]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")
data_webstat = pd.read_csv("./data/t1_webstat.csv")

is_callcenter - был ли совершен звонок в коллцентр по итогам сессии (нужно предсказать)

order_id - id заказа

create_time - время создания заказа

good_id - id товара

price - цена товара

utm_medium - тип трафика

utm_source - источник трафика

sessionkey_id - id сессии пользователя

category_id - id категории товара

parent_id - id родительской категории для категории товара

root_id - id "базовой" категории товара

rating_value - рейтинг товара

rating_count - число оценок товара

description_length - длина описания товара

goods_qty - число товаров в заказе

model_id - ???

is_moderated - ???

pics_qty - ???

model_create_time - ???

sessionkey_id - id сессии пользователя

date_time - timestamp захода на страницу определенного типа

page_type - тип просмотренной страницы

pageview_number - порядковый номер страницы среди всех страниц, отсортированных по времени посещения внутри сессии

pageview_duration_sec - время просмотра страницы

product_in_sale - ???

In [145]:
data_train["create_time_ts"] = None
data_test["create_time_ts"] = None
data_train["model_create_time_ts"] = None
data_test["model_create_time_ts"] = None

from datetime import datetime
# convert string to date
data_train.create_time_ts = data_train.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.create_time_ts = data_test.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

data_train.loc[~data_train.model_create_time.isna(), "model_create_time_ts"] = data_train.loc[~data_train.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.loc[~data_test.model_create_time.isna(), "model_create_time_ts"] = data_test.loc[~data_test.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [146]:
del data_train["create_time"]
del data_train["model_create_time"]
del data_test["create_time"]
del data_test["model_create_time"]

In [151]:
data_train.model_create_time_ts = data_train.model_create_time_ts.fillna(data_train.create_time_ts)
data_test.model_create_time_ts = data_test.model_create_time_ts.fillna(data_test.create_time_ts)

In [171]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104595 entries, 0 to 104594
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   order_id              104595 non-null  int64         
 1   good_id               104595 non-null  int64         
 2   price                 104595 non-null  int64         
 3   utm_medium            104595 non-null  int64         
 4   utm_source            94145 non-null   float64       
 5   sessionkey_id         104595 non-null  int64         
 6   category_id           104595 non-null  int64         
 7   parent_id             104595 non-null  int64         
 8   root_id               104595 non-null  int64         
 9   model_id              104595 non-null  int64         
 10  is_moderated          104595 non-null  int64         
 11  rating_value          33741 non-null   float64       
 12  rating_count          51613 non-null   float64       
 13 

In [172]:
data_train.isna().sum().sort_values()

order_id                    0
is_callcenter               0
pics_qty                    0
goods_qty                   0
description_length          0
is_moderated                0
create_time_ts              0
root_id                     0
model_id                    0
category_id                 0
sessionkey_id               0
utm_medium                  0
price                       0
good_id                     0
parent_id                   0
model_create_time_ts        0
utm_source              10450
rating_count            52982
rating_value            70854
dtype: int64

In [185]:
data_train[data_train["sessionkey_id"] == 122260953]

,order_id,good_id,price,utm_medium,utm_source,sessionkey_id,category_id,parent_id,root_id,model_id,is_moderated,rating_value,rating_count,description_length,goods_qty,pics_qty,is_callcenter,create_time_ts,model_create_time_ts
82702,1256366,67593020,1690,2,NaN,122260953,3009,1229,1183,24908006,1,NaN,0.0,1556,15,5,1,1975-12-18 16:06:50,1975-06-01 15:31:52


In [184]:
data_webstat[data_webstat["sessionkey_id"] == 122260953]

,sessionkey_id,date_time,page_type,pageview_number,pageview_duration_sec,category_id,model_id,good_id,price,product_in_sale
2565,122260953,1975-12-18 15:42:20.460,1,2,115.0,3009.0,28805855.0,60138693.0,1823.0,1.0
2566,122260953,1975-12-18 15:49:16.020,1,6,55.0,3009.0,17075020.0,68428948.0,1908.0,1.0
2567,122260953,1975-12-18 15:50:11.337,2,7,116.0,3009.0,NaN,NaN,NaN,NaN
2568,122260953,1975-12-18 15:55:09.723,1,10,47.0,3009.0,12824899.0,62509826.0,1879.0,1.0
2569,122260953,1975-12-18 15:55:56.023,2,11,90.0,3009.0,NaN,NaN,NaN,NaN
511448,122260953,1975-12-18 15:40:32.233,2,1,108.0,3009.0,NaN,NaN,NaN,NaN
511449,122260953,1975-12-18 15:59:06.263,1,14,20.0,3009.0,24908006.0,67593020.0,1690.0,1.0
531199,122260953,1975-12-18 15:46:49.297,2,5,147.0,3009.0,NaN,NaN,NaN,NaN
531200,122260953,1975-12-18 15:52:07.897,1,8,59.0,3009.0,28805855.0,60138693.0,1823.0,1.0
531201,122260953,1975-12-18 15:53:06.353,2,9,123.0,3009.0,NaN,NaN,NaN,NaN


In [197]:
del data_train["order_id"]

In [198]:
data_train

,good_id,price,utm_medium,utm_source,sessionkey_id,category_id,parent_id,root_id,model_id,is_moderated,rating_value,rating_count,description_length,goods_qty,pics_qty,is_callcenter,create_time_ts,model_create_time_ts
0,9896348,753,5,8.0,123777004,139,133,124,123517,1,5.0,6.0,1204,6,2,1,1975-12-26 09:30:08,1971-04-14 00:15:20
1,9896348,753,1,2.0,123781654,139,133,124,123517,1,5.0,6.0,1204,6,2,0,1975-12-26 10:28:57,1971-04-14 00:15:20
2,9896348,753,2,3.0,123591002,139,133,124,123517,1,5.0,6.0,1204,6,2,1,1975-12-25 11:24:28,1971-04-14 00:15:20
3,9896348,753,1,1.0,123832302,139,133,124,123517,1,5.0,6.0,1204,6,2,1,1975-12-26 14:16:06,1971-04-14 00:15:20
4,9896348,753,3,56.0,123881603,139,133,124,123517,1,5.0,6.0,1204,6,2,0,1975-12-26 18:21:47,1971-04-14 00:15:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104590,68657110,1497,4,1.0,121795335,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1,1975-12-16 12:11:32,1976-01-23 00:08:20
104591,60391507,1354,1,2.0,114277105,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1,1975-11-08 11:29:50,1976-01-23 00:08:20
104592,66971400,1452,2,NaN,114972131,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,0,1975-11-12 09:14:35,1976-01-23 00:08:20
104593,60391498,1433,4,2.0,114672813,2873,1229,1183,35231362,1,4.0,3.0,1183,12,1,1,1975-11-10 17:28:07,1976-01-23 00:08:20


In [204]:
print("Рейтинг товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["rating_value"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["rating_value"].mean())

Рейтинг товаров, по которым позвонили:  4.610021786492375 , по которым не звонили 4.670367247517332


In [205]:
print("Длина описания товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["description_length"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["description_length"].mean())

Длина описания товаров, по которым позвонили:  572.92983638373 , по которым не звонили 573.338671921299


In [207]:
data_train.isna().sum().sort_values()

good_id                     0
is_callcenter               0
pics_qty                    0
goods_qty                   0
description_length          0
is_moderated                0
create_time_ts              0
model_id                    0
parent_id                   0
category_id                 0
sessionkey_id               0
utm_medium                  0
price                       0
root_id                     0
model_create_time_ts        0
utm_source              10450
rating_count            52982
rating_value            70854
dtype: int64

In [220]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [221]:
features = ["pics_qty", "goods_qty", "description_length", "is_moderated", "model_id", "parent_id", "category_id", "utm_medium", "price", "root_id"]

In [222]:
X_train, X_test, y_train, y_test = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.2, random_state=42)

In [223]:
model = CatBoostClassifier(eval_metric='AUC', verbose=100)

In [224]:
model.fit(X_train, y_train,eval_set=(X_test, y_test), use_best_model=True)

Learning rate set to 0.094578
0:	test: 0.5916887	best: 0.5916887 (0)	total: 69.5ms	remaining: 1m 9s
100:	test: 0.6304607	best: 0.6304759 (95)	total: 1.18s	remaining: 10.5s
200:	test: 0.6317850	best: 0.6318103 (181)	total: 2.3s	remaining: 9.16s
300:	test: 0.6309411	best: 0.6318103 (181)	total: 3.39s	remaining: 7.86s
400:	test: 0.6310831	best: 0.6318103 (181)	total: 4.4s	remaining: 6.57s
500:	test: 0.6300656	best: 0.6318103 (181)	total: 5.48s	remaining: 5.46s
600:	test: 0.6295612	best: 0.6318103 (181)	total: 6.52s	remaining: 4.33s
700:	test: 0.6290861	best: 0.6318103 (181)	total: 7.58s	remaining: 3.23s
800:	test: 0.6287054	best: 0.6318103 (181)	total: 8.55s	remaining: 2.13s
900:	test: 0.6283776	best: 0.6318103 (181)	total: 9.53s	remaining: 1.05s
999:	test: 0.6276301	best: 0.6318103 (181)	total: 10.5s	remaining: 0us

bestTest = 0.6318102553
bestIteration = 181

Shrink model to first 182 iterations.


In [225]:
predictions = model.predict(data_test[features])

In [227]:
id_column = data_test["order_id"]
result_column = predictions
dataset = pd.DataFrame({'order_id': id_column, 'is_callcenter': list(result_column)}, columns=['order_id', 'is_callcenter'])

In [228]:
dataset.to_csv("sample_submission.csv", index=False)